<a href="https://colab.research.google.com/github/modhudeb/determine-Pneumonia-Xray/blob/main/2_Xray_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
from skimage import io, transform

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.metrics import classification_report

# Loading data

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data/files/xrayImg.csv", index_col='Num')

In [ ]:
df = data.copy()

In [ ]:
df['label'].value_counts()

1    3806
0    1349
Name: label, dtype: int64

In [ ]:
testData = pd.read_csv("/content/drive/MyDrive/data/files/xray_test.csv", index_col='Num')
tdf = testData.copy()

# EDA

Handling imbalanced data

In [ ]:
smote =  SMOTE(random_state=42, k_neighbors=5, n_jobs=-1)
features, targets = smote.fit_resample(df.drop(columns=['label']), df['label'])

In [ ]:
targets.value_counts()

0    3806
1    3806
Name: label, dtype: int64

# Algorithms

In [ ]:
# training full dataset
mlpF = MLPClassifier(hidden_layer_sizes=(128,32,16), random_state=42)
mlpF.fit(features, targets)

MLPClassifier(hidden_layer_sizes=(128, 32, 16), random_state=42)

In [ ]:
rep1 = classification_report(tdf['label'], mlpF.predict(tdf.drop(columns=['label'])), target_names=['Normal', 'Pneumonia'])

print(rep1)

              precision    recall  f1-score   support

      Normal       0.98      0.48      0.65       234
   Pneumonia       0.76      0.99      0.86       390

    accuracy                           0.80       624
   macro avg       0.87      0.74      0.76       624
weighted avg       0.84      0.80      0.78       624



Checking SVM with full dataset

In [ ]:
svF = SVC(random_state=42, probability=True)
svF.fit(features, targets)

SVC(probability=True, random_state=42)

In [ ]:
rep2 = classification_report(tdf['label'], svF.predict(tdf.drop(columns=['label'])), target_names=['Normal', 'Pneumonia'])

print(rep2)

              precision    recall  f1-score   support

      Normal       0.97      0.66      0.79       234
   Pneumonia       0.83      0.99      0.90       390

    accuracy                           0.87       624
   macro avg       0.90      0.83      0.85       624
weighted avg       0.88      0.87      0.86       624



# Testing with single image

In [ ]:
# class for converting an Image to array

class Handler:
  # function to crop image
  def __imgCrop(img):
    img = img[150:(img.shape[0]-150), 150:(img.shape[1]-150)]
    return img

  # function to resize and transform to 1D array
  def preprocess(self, path:str):
    img = io.imread(path, as_gray=True)
    img = Handler.__imgCrop(img)
    img = transform.resize(img, (150,150))
    return img.ravel()
  


In [ ]:
imgArr = Handler().preprocess("BACTERIA-545831-0001.jpeg")
imgArr = np.array([imgArr],)
pred = svF.predict_proba(imgArr)
ind = np.argmax(pred)
print("NORMAL\n\n") if ind == 0 else print("PNEUMONIA\n\n")

PNEUMONIA




/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from sklearn.pipeline import Pipeline
from joblib import dump

In [ ]:
ppl = Pipeline([('svm', svF)])

In [ ]:
ppl.score(tdf.drop(columns=['label']), tdf['label'])

0.8669871794871795

In [ ]:
dump(ppl, 'Xray_SVM')

['Xray_SVM']

No Model is giving expected accuracy, so we will try CNN 